In [1]:
#Import of Python available libraries
import sculib

mpi = sculib.scu()
#mpi.ip = '10.96.64.10'
mpi.ip = '10.96.254.85'

In [4]:
def mpi_t688(scu):
    #Original 1
    # Suggested Henk 2) iii

    #Feed Indexer tests [316-000000-043] 
    print('Test 6.8.8')

    #Revisions:
    #0.1 initial rewrite
    #0.2 loop through all elevations, sugest change lower el from 15 to 20, move az and el independantly
    #0.3 updates from Henk 4/9/2020
    #0.4 convert into function, import sculib as class and create mpi object to work with

    #Common setup
    config_name = 'HN_INDEX_TEST'
  
    loop1 = 10
    loop2 = 5

    #elevations = [15, 45, 85] lower limit not accessible
    az_start = -90
    az_speed = 3
    el_start = 20 #15
    el_speed = 0.1
    #elevations = [20, 45, 85]
    elevations = [20]
    wait5 = 5
    wait10 = 10
    wait100 = 100
    sampling = 200
    #test only remove these
    '''
    wait5 = 1
    wait10 = 1
    wait100 = 1
    sampling = 5000
    scu.debug=True
    '''
    #Feed Indexer: Response & Repeatability (6.8.8)
    '''
        a. Feed Indexer: Response & Repeatability (6.8.8)
        1) Manually move the Dish to 15° Elevation, any Azimuth angle (suitable for the current site conditions) and FI at SPF Band5c and de-activate all axes.
        2) Manually start the LDS data recording
        3) Start the ACU data recording on the SCU using logging configuration TBD 
        4) Command the FI to the SPF B1 position,
            ◦  wait for 40 seconds to complete the move
        5) Command the FI to the SPF B2 position, 
            ◦ wait for 40 seconds to complete the move
        6) Command the FI to the SPF Band 5c position,
            ◦ wait for 100 seconds to complete the move
        8) Command the FI to SPF B1 position, 
            ◦ wait 50 seconds to complete the move 
        9) Command the FI to SPF B5c position, 
            ◦ wait 100 seconds to complete the move and to stabilise
        10) Command the FI to SPF B2 position, 
            ◦ wait 50 seconds to complete the move 
        11) Command the FI to SPF B5c position, 
            ◦ wait 100 seconds to complete the move and to stabilise
        12) Repeat steps 7) to 10) for a total of 10 cycles
        13) Command the FI to SPF B1 position, 
            ◦ wait 40 seconds to complete the move 
        14) Command the FI to SPF B2 position, 
            ◦ wait 100 seconds to complete the move 
        15) Command the FI to SPF B1 position, 
            ◦ wait 100 seconds to complete the move 
        16) Repeat steps 13) to 14)  for a total of 5 cycles 
        17) Command the FI to SPF B5c position, 
            ◦ wait 40 seconds to complete the move
        18) Stop the ACU data recording on the SCU
        19) Manually stop the LDS data recordings
        20) Extract the data on the SCU @ 5Hz sampling
        21) Rename the file to TBD (eg. 6p8p8_i.csv)
        22) Repeat the sequence 1) to 19) at 45° Elevation.
        23) Repeat the sequence 1) to 19) at 85° Elevation.
    (Total duration ~ 1h per elevation angle => 3h)
    '''
    #Step 1
    scu.activate_dmc()
    scu.wait_duration(wait10*3)

    #Move to starting az
    scu.abs_azimuth(az_start, az_speed)
    scu.wait_duration(wait10)

    #Move to starting el
    scu.abs_elevation(el_start, el_speed)
    scu.wait_duration(wait10*3)

    #Move to starting band
    band = 'Band 5c'
    scu.move_to_band(band)
    scu.wait_duration(wait10*4)

    for el_angle in elevations:
        #Step 2
        print('Start LDS recording here:')
        #input()

        print('Select elevation angle: {:.3f}  \n'.format(el_angle))
        scu.abs_elevation(el_angle, el_speed)
        scu.wait_duration(wait10*4)

        #Step 3
        if scu.logger_state() == 'RECORDING':
            print('WARNING, already recording - attempting to stop and start a fresh logger')
            scu.stop_logger()  
            scu.wait_duration(wait5)
        if scu.logger_state() == 'STOPPED':
            print('Starting logger with config: {}'.format(config_name))
            scu.start_logger(config_name)
            scu.wait_duration(wait5)

        #Step 4
        band = 'Band 1'
        scu.move_to_band(band)
        scu.wait_duration(wait10*5)

        #Step 5
        band = 'Band 2'
        scu.move_to_band(band)
        scu.wait_duration(wait10*5)

        #Step 6
        band = 'Band 5c'
        scu.move_to_band(band)
        scu.wait_duration(wait100)

        #Step 7 deleted but start of loop


        #Loop Step 7 to 10, 10 times
        i=0
        while i < loop1:
            i+=1
            print('Loop 1 - Step 7 to 10: {:.0f} of {:.0f}'.format(i, loop1))

            #Step 8
            band = 'Band 1'
            scu.move_to_band(band)
            scu.wait_duration(wait10*5)

            #Step 9
            band = 'Band 5c'
            scu.move_to_band(band)
            scu.wait_duration(wait100)

            #Step 10
            band = 'Band 2'
            scu.move_to_band(band)
            scu.wait_duration(wait10*5)

            #Step 11
            band = 'Band 5c'
            scu.move_to_band(band)
            scu.wait_duration(wait100)
        #Step 12 loop end

        #Step 13
        band = 'Band 1'
        scu.move_to_band(band)
        scu.wait_duration(wait10*5)

        #Loop Step 14 to 15, 5 times
        i=0
        while i < loop2:
            i += 1
            print('Loop 2 - Step 14 to 15: {:.0f} of {:.0f}'.format(i, loop2))

            #Step 14
            band = 'Band 2'
            scu.move_to_band(band)
            scu.wait_duration(wait100)

            #Step 15
            band = 'Band 1'
            scu.move_to_band(band)
            scu.wait_duration(wait100)

            #Step 16 - loop end cycle 14 to 15

        ####    print('End elevation: {:.3f}'.format(el_angle))
        #Step 17
        band = 'Band 5c'
        scu.move_to_band(band)
        scu.wait_duration(wait10*5)

        #Step 18
        scu.stop_logger()
        scu.wait_duration(wait5)
        print(scu.logger_state())

        #Step 19
        print('Stop LDS recording now :')    
        #input()
        #Step 20
        print('Extract the data on the SCU @ 5Hz sampling')
        scu.save_session('6p8p8_i_elev_' + str(el_angle) + '_', interval_ms = sampling)
        #Step 21
        print('Rename the file to TBD (eg. 6p8p8_i.csv)')
        #Step 22
        #print('Repeat the sequence 1) to 19) at 45° Elevation.')
        #Step 23
        #print('Repeat the sequence 1) to 19) at 85° Elevation.')
    print('Dropping elevation')
    scu.abs_elevation(el_start, el_speed)
    scu.wait_duration(wait10*3)
    print('Done')
    scu.deactivate_dmc()
    scu.wait_duration(wait10*3)


In [5]:
mpi_t688(mpi)


Test 6.8.8
activate dmc
***Feedback: http://10.96.254.85:8080/devices/command b'{"path": "acu.dish_management_controller.activate"}'
OK 200
***Text returned:
{"uuid":"4e5c8293-b8ea-4c0e-a074-37acf8b992c0"}

wait for 3.0s done *
abs az: -90.000 vel: 3.000
***Feedback: http://10.96.254.85:8080/devices/command b'{"path": "acu.azimuth.slew_to_abs_pos", "params": {"new_axis_absolute_position_set_point": -90, "new_axis_speed_set_point_for_this_move": 3}}'
OK 200
***Text returned:
{"uuid":"a3bc85bf-695a-4aac-8c61-2a04065e3ce0"}

wait for 1.0s done *
abs el: 20.000 vel: 0.100
***Feedback: http://10.96.254.85:8080/devices/command b'{"path": "acu.elevation.slew_to_abs_pos", "params": {"new_axis_absolute_position_set_point": 20, "new_axis_speed_set_point_for_this_move": 0.1}}'
OK 200
***Text returned:
{"uuid":"e4fd81ba-30f4-47cb-a817-087ff082cae9"}

wait for 3.0s done *
move to band: Band 5c
***Feedback: http://10.96.254.85:8080/devices/command b'{"path": "acu.dish_management_controller.move_to_b

wait for 5.0s done *
move to band: Band 5c
***Feedback: http://10.96.254.85:8080/devices/command b'{"path": "acu.dish_management_controller.move_to_band", "params": {"action": 7}}'
OK 200
***Text returned:
{"uuid":"ebe60491-872f-486e-a64b-bfa896527c68"}

wait for 1.0s done *
move to band: Band 2
***Feedback: http://10.96.254.85:8080/devices/command b'{"path": "acu.dish_management_controller.move_to_band", "params": {"action": 2}}'
OK 200
***Text returned:
{"uuid":"231ad91c-bfed-4624-877d-347b8e952dca"}

wait for 5.0s done *
move to band: Band 5c
***Feedback: http://10.96.254.85:8080/devices/command b'{"path": "acu.dish_management_controller.move_to_band", "params": {"action": 7}}'
OK 200
***Text returned:
{"uuid":"acb84fa5-5588-48e3-9b30-a30379763379"}

wait for 1.0s done *
Loop 1 - Step 7 to 10: 7 of 10
move to band: Band 1
***Feedback: http://10.96.254.85:8080/devices/command b'{"path": "acu.dish_management_controller.move_to_band", "params": {"action": 1}}'
OK 200
***Text returned:


***Feedback: http://10.96.254.85:8080/datalogging/stop b'{}'
OK 200
***Text returned:
{"uuid":null,"startTime":"2020-09-14T13:12:51.189442500Z","stopTime":"2020-09-14T13:15:26.003720700Z","duration":"00:02:34","configId":24,"fullFilePath":".\\datalogging\\2020\\9\\14___13_12_51","name":"HN_INDEX_TEST"}

wait for 1.0s done *
logger state 
***Feedback: http://10.96.254.85:8080/datalogging/currentState None
OK 200
***Text returned:
{"state":"STOPPED"}

STOPPED
Stop LDS recording now :
Extract the data on the SCU @ 5Hz sampling
Attempt export and save of session: last at rate 5000 ms
logger sessions 
***Feedback: http://10.96.254.85:8080/datalogging/sessions None
OK 200
***Text returned:
[{"uuid":1,"startTime":"2020-08-24T12:28:09.219205300Z","stopTime":"2020-08-24T12:29:10.925214900Z","duration":"00:01:01","configId":3,"fullFilePath":".\\datalogging\\2020\\8\\24___12_28_09","name":"index_logger.txt"},{"uuid":2,"startTime":"2020-08-24T12:32:20.466192700Z","stopTime":"2020-08-24T12:32:31.59

***Feedback: http://10.96.254.85:8080/datalogging/exportSession?id=88&interval_ms=5000 None
OK 200
***Text returned:
StartTime;2020-09-14T13:12:51.189442500Z
EndTime;2020-09-14T13:15:26.003720700Z
;acu.time.act_time_source;acu.time.internal_time;acu.time.external_ptp;acu.general_management_and_controller.state;acu.general_management_and_controller.feed_indexer_pos;acu.azimuth.state;acu.azimuth.p_set;acu.azimuth.p_act;acu.azimuth.v_act;acu.elevation.state;acu.elevation.p_set;acu.elevation.p_act;acu.elevation.v_act;acu.feed_indexer.state;acu.feed_indexer.p_set;acu.feed_indexer.p_shape;acu.feed_indexer.p_act;acu.feed_indexer.v_shape;acu.feed_indexer.v_act;acu.feed_indexer.motor_1.actual_velocity;acu.feed_indexer.motor_2.actual_velocity;acu.feed_indexer.motor_1.actual_torque;acu.feed_indexer.motor_2.actual_torque;acu.general_management_and_controller.act_power_consum;acu.general_management_and_controller.power_factor;acu.general_management_and_controller.voltage_phase_1;acu.general_managem